In [3]:
# %load 8_puzzle_game.py
"""
Artifical Intelligence - MSOE
Author - Gagan Daroach <gagandaroach@gmail.com>
April 26 2019

An implementation of the Puzzle8 Game. 
This file is part of my coursework during my undergrad at Milwaukee School of Engineering.
The purpose of this file is to be used as a illustration and learning tool
    for the hill climbing and a* search techniques with different heuristics.
"""

import numpy as np
import random
import pprint as pp
from enum import Enum, auto


class Direction(Enum):
    RIGHT = auto()
    LEFT = auto()
    DOWN = auto()
    UP = auto()


class Puzzle8:
    """
    Position Locations
    [ [0. 1. 2.]
      [3. 4. 5.]
      [6. 7. 8.] ]
    """
    def __init__(self, pattern = "012345678"):
        self.grid = self.emptyGrid()
        self.load_pattern(pattern)

    def emptyGrid(self):
        return np.zeros((3, 3))

    def load_pattern(self, pattern):
        """pattern format: 012345678"""
        if len(pattern) != 9:
            print("Error - Pattern %s was not the correct length." % pattern)
        else:
            for pos in range(9):
                self.set_tile_at(pos, pattern[pos])

    def dump_pattern(self):
        pattern = ""
        for index in range(9):
            pattern += str(int(self.get_tile_at(index)))
        return pattern

    def __str__(self):
        return ' Grid Printout:\n' + str(self.grid)

    def swap_tiles(self, pos1, pos2):
        """swaps the values of two tiles"""
        temp = self.get_tile_at(pos1)
        self.set_tile_at(pos1, self.get_tile_at(pos2))
        self.set_tile_at(pos2, temp)

    def get_tile_at(self, pos):
        coordinates = self.pos_to_coordinate(pos)
        return int(self.grid[coordinates[1]][coordinates[0]])

    def set_tile_at(self, pos, val):
        coordinates = self.pos_to_coordinate(pos)
        self.grid[coordinates[1]][coordinates[0]] = val

    def pos_to_coordinate(self, pos):
        y = self.pos_row(pos)
        x = self.pos_col(pos)
        return (x, y)

    def pos_row(self, pos):
        return pos // 3

    def pos_col(self, pos):
        return pos % 3

    def find_missing_tile(self):
        return self.find_tile(0)

    def find_tile(self, val):
        for pos in range(9):
            looking_at = self.get_tile_at(pos)
            if looking_at == val:
                return pos
        return None

    def move_missing_tile(self, direction):
        pos = self.find_missing_tile()

        if direction == Direction.UP:
            if (self.pos_row(pos) == 0):
                return False
            else:
                self.swap_tiles(pos, pos - 3)
                return True
        elif direction == Direction.RIGHT:
            if (self.pos_col(pos) == 2):
                return False
            else:
                self.swap_tiles(pos, pos + 1)
                return True
        elif direction == Direction.LEFT:
            if (self.pos_col(pos) == 0):
                return False
            else:
                self.swap_tiles(pos, pos - 1)
                return True
        else:
            if (self.pos_row(pos) == 2):
                return False
            else:
                self.swap_tiles(pos, pos + 3)
                return True

    def randomize_puzzle(self, num_random_moves = 15, debug = False):
        count = 0
        while count < num_random_moves:
            random_direction = random.choice(list(Direction))
            if (self.move_missing_tile(random_direction)):
                if debug: print(self)
                count += 1

    def calculate_misplaced_tiles(self, solution_pattern, debug = False):
        num_correct = 0
        for pos in range(9):
            if int(self.get_tile_at(pos)) == int(solution_pattern[pos]):
                if debug: print(solution_pattern[pos], self.get_tile_at(pos))
                num_correct += 1
        return 9 - num_correct

    def calculate_manhattan_distance(self, solution_pattern, debug = False):
        man_distance = 0
        for pos in range(9):
            my_tile = int(self.get_tile_at(pos))
            sol_tile = int(solution_pattern[pos])
            my_pos = self.find_tile(my_tile)
            sol_pos = self.find_tile(sol_tile)
            distance = self.pos_distance(my_pos, sol_pos)
            man_distance += distance
        return man_distance

    def pos_distance(self, pos1, pos2):
        coord1 = self.pos_to_coordinate(pos1)
        coord2 = self.pos_to_coordinate(pos2)
        deltaX = abs(coord1[0] - coord2[0])
        deltaY = abs(coord1[1] - coord2[1])
        return deltaX + deltaY

In [6]:
puzzle = Puzzle8()
puzzle.randomize_puzzle()
puzzle.dump_pattern()

'301452678'

In [7]:
pattern = _
print(pattern)

301452678


```
Discrete Space Hill Climbing Algorithm
   currentNode = startNode;
   loop do
      L = NEIGHBORS(currentNode);
      nextEval = -INF;
      nextNode = NULL;
      for all x in L 
         if (EVAL(x) > nextEval)
              nextNode = x;
              nextEval = EVAL(x);
      if nextEval <= EVAL(currentNode)
         //Return current node since no better neighbors exist
         return currentNode;
      currentNode = nextNode;
```

In [47]:
def calculate_possible_heuristics(current_pattern, solution_pattern):
    heuristics = {}
    for enum in Direction:
        duplicate = Puzzle8(current_pattern)
        duplicate.move_missing_tile(enum)
        heuristic_value = duplicate.calculate_misplaced_tiles(solution_pattern)
        heuristics[enum] = heuristic_value
    return heuristics

In [50]:
calculate_possible_heuristics("012345678", "012345678")

{<Direction.RIGHT: 1>: 2,
 <Direction.LEFT: 2>: 0,
 <Direction.DOWN: 3>: 2,
 <Direction.UP: 4>: 0}

In [51]:
def determine_best_possible_heuristic(heuristics):
    lowest_enum_value = 100000
    best_heuristic = None
    for enum in Direction:
        if heuristics[enum] < lowest_enum_value:
            best_heuristic = enum
            lowest_enum_value = heuristics[enum]
    return best_heuristic

In [52]:
poss_heuristics = calculate_possible_heuristics("012345678", "012345678")
determine_best_possible_heuristic(poss_heuristics)

<Direction.LEFT: 2>

In [57]:
def climb_that_hill(init_pattern, sol_pattern = "012345678", max_moves = 100):
    print("Hill Climb Search:\n\tinit pattern: %s\n\ttarget pattern: %s" % (init_pattern, sol_pattern))
    heuristic = 1
    num_moves = 0
    current_pattern = init_pattern
    while heuristic != 0 and num_moves < max_moves:
        poss_heuristics = calculate_possible_heuristics(current_pattern, sol_pattern)
        enum_best_move = determine_best_possible_heuristic(poss_heuristics)
        puzzle = Puzzle8(current_pattern)
        puzzle.move_missing_tile(enum_best_move)
        heuristic = puzzle.calculate_misplaced_tiles(sol_pattern)
        current_pattern = puzzle.dump_pattern()
        num_moves += 1
    return num_moves
        

In [58]:
climb_that_hill(pattern)

Hill Climb Search:
	init pattern: 301452678
	target pattern: 012345678


5